Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [211]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Przygotowanie macierzy interakcji

In [212]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
productsDataPath = '../notebooks/data/v2/products.jsonl'
usersDataPath = '../notebooks/data/v2/users.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)
usersDF = pd.read_json(usersDataPath, lines=True)

df = sessionsDF.drop(columns=["timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1

#TODO Below split does not make any sense. Proper split need to be aplied for final build, but this one works as fast split for testing.
#BEGIN OF SHIT
train, test = train_test_split(df, test_size=0.2, stratify=df['user_id'])

#idea is to make sure that all data in test set has at least two entries from given session. Data that not satisfies that criteria are appended to train set.
s = test['session_id'].value_counts()
res = test[test['session_id'].map(s) >= 2]
ret = test[test['session_id'].map(s) == 1]
test.sort_values(by='session_id', inplace=True)
res.sort_values(by='session_id', inplace=True)
# display(test)
# display(res)
#END OF SHIT

train.append(ret).sort_values(by='session_id', inplace=True)
interactionMatrixDF = pd.pivot_table(train, index="user_id", columns="product_id", values="count", aggfunc=np.sum, fill_value=0)
display(interactionMatrixDF)


C:\Users\Bartosz\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103,0,1,3,14,1,2,2,4,2,9,...,0,0,2,0,1,2,5,1,0,7
104,0,0,5,4,4,7,0,5,3,4,...,1,1,0,0,1,1,6,1,5,1
105,0,0,7,9,4,5,0,3,1,4,...,0,1,2,0,2,1,6,2,0,0
106,2,1,1,5,1,3,2,3,1,3,...,0,0,2,3,1,1,4,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,3,1,4,11,7,1,0,3,5,5,...,0,1,0,2,0,1,8,1,3,3
298,0,0,6,1,3,3,0,5,1,1,...,1,0,0,0,0,1,8,3,0,0
299,0,0,2,0,0,0,0,1,0,1,...,0,0,1,0,2,1,3,1,0,0


Normalizacja wartości w macierzy interakcji.

In [213]:
#TODO do not know if scaler is required at all
interactionMatrixDF = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(interactionMatrixDF))
display(interactionMatrixDF)

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
0,0.000000,0.00,0.071429,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.166667,0.000,0.000000
1,0.000000,0.25,0.214286,0.538462,0.076923,0.153846,0.4,0.333333,0.222222,0.529412,...,0.0,0.000000,0.50,0.00,0.142857,0.50,0.217391,0.166667,0.000,1.000000
2,0.000000,0.00,0.357143,0.153846,0.307692,0.538462,0.0,0.416667,0.333333,0.235294,...,0.5,0.142857,0.00,0.00,0.142857,0.25,0.260870,0.166667,0.625,0.142857
3,0.000000,0.00,0.500000,0.346154,0.307692,0.384615,0.0,0.250000,0.111111,0.235294,...,0.0,0.142857,0.50,0.00,0.285714,0.25,0.260870,0.333333,0.000,0.000000
4,0.333333,0.25,0.071429,0.192308,0.076923,0.230769,0.4,0.250000,0.111111,0.176471,...,0.0,0.000000,0.50,0.75,0.142857,0.25,0.173913,0.333333,0.000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.500000,0.25,0.285714,0.423077,0.538462,0.076923,0.0,0.250000,0.555556,0.294118,...,0.0,0.142857,0.00,0.50,0.000000,0.25,0.347826,0.166667,0.375,0.428571
196,0.000000,0.00,0.428571,0.038462,0.230769,0.230769,0.0,0.416667,0.111111,0.058824,...,0.5,0.000000,0.00,0.00,0.000000,0.25,0.347826,0.500000,0.000,0.000000
197,0.000000,0.00,0.142857,0.000000,0.000000,0.000000,0.0,0.083333,0.000000,0.058824,...,0.0,0.000000,0.25,0.00,0.285714,0.25,0.130435,0.166667,0.000,0.000000
198,0.333333,1.00,0.500000,0.269231,0.230769,0.307692,0.0,0.333333,0.222222,0.352941,...,1.0,0.142857,0.00,0.00,0.571429,0.75,0.347826,0.333333,0.000,0.571429


Utworzenie słownika dla szybszego wyszukiwania.

In [214]:
idxNameDict = pd.Series(productsDF["product_id"].values, index=productsDF.index).to_dict()
idxUserDict = pd.Series(usersDF["user_id"].values, index=usersDF.index).to_dict()

Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [215]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate user latent features
userSVD = TruncatedSVD(n_components=latentFactors)
userFeatures = userSVD.fit_transform(interactionMatrixDF) + epsilon

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose()) + epsilon #transpose because items are columns


pd.DataFrame(itemFeatures)
pd.DataFrame(userFeatures)


,0,1,2,3,4,5,6,7,8,9
0,0.337585,0.012785,-0.083225,0.107005,0.031841,-0.024249,0.001669,-0.065308,-0.027275,0.073110
1,4.402698,0.494786,-0.274655,-1.017609,-0.059812,-0.195893,-0.838336,-0.168983,-0.386076,0.327676
2,3.683884,0.515077,-0.584847,0.582949,0.279331,0.075409,0.937740,-0.032088,-0.143856,-0.460887
3,5.413924,0.410812,2.195497,-0.398554,-0.392959,-0.790216,-0.462373,1.287740,-0.165992,-0.142460
4,3.625618,-0.370262,0.541847,-0.273488,0.032092,-0.298501,0.505936,0.293242,0.898874,0.051817
...,...,...,...,...,...,...,...,...,...,...
195,4.680978,-0.056340,0.653497,0.102030,-0.352947,0.191119,0.316954,-1.394408,0.713301,-0.744553
196,2.725414,0.199057,-0.360775,-0.501666,-0.542431,-0.176547,0.164493,0.393700,0.260029,-0.114049
197,0.992823,-0.237015,0.141057,-0.021192,-0.143743,0.068221,-0.183253,0.067723,0.175613,0.076345
198,3.665758,-1.093140,-0.900417,-0.833453,0.136941,-0.100429,-0.455480,-0.900167,0.314543,-0.500524


In [217]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=8, n_init=1000)

model = kmeans.fit_predict(userFeatures)
modelDF = pd.DataFrame(model).rename(columns={0 : 'group'}, index=idxUserDict)
modelDF



,group
102,5
103,7
104,3
105,6
106,3
...,...
297,2
298,1
299,5
300,7


Budowanie profili użytkowników  
_TESTY_

In [218]:
separator = ';'
newGroups = ['Gry komputerowe', 'Gry na konsole', 'Sprzęt RTV', 'Komputery', 'Telefony i akcesoria']

def castCategoryPath(categoryPath):
    categories = categoryPath.split(separator)
    foundGroups = [group for group in newGroups if group in categories]
    if len(foundGroups) != 1:
        raise RuntimeError('wrong group cast: {}'.format(foundGroups))
    return foundGroups[0]

In [219]:
def build_user_profile(userID: int, merged: pd.DataFrame) -> dict:
    profile = {}
    userActDF = merged.loc[merged['user_id'] == userID].drop(columns=['user_id'])

    for group in newGroups:
        g = userActDF.loc[userActDF['category_path'] == group].drop(columns=['category_path']).value_counts().to_frame().rename(columns={0 : 'count'})
        g = g[g['count'] >= g['count'].quantile(0.9)]
        profile[group] = g.to_dict()
    return profile


In [220]:
merged = pd.merge(sessionsDF, productsDF, how='inner', on='product_id')
merged.drop(columns=['session_id', "timestamp", "event_type", "offered_discount", "purchase_id", 'product_name', 'price', 'user_rating'], inplace=True)
merged['category_path'] = merged['category_path'].apply(castCategoryPath)

usersID = usersDF['user_id'].unique()
profilDict = {}

for userID in usersID:
    profilDict[userID] = build_user_profile(userID, merged)

_KONIEC TESTÓW_

Przykład predykcji  
Użytkownik o id 102 przegląda produkt z kategorii *Gry komputerowe*

In [221]:
input = [102, 'Gry komputerowe']

group = modelDF.loc[modelDF['group'] == modelDF.iloc[102]['group']].index
dfListPre = []

for user in group:
    dfListPre.append(pd.DataFrame(profilDict[user][input[1]]))

dfList = [df.reset_index().rename(columns={'level_0' : 'product_id'}) for df in dfListPre]

df = pd.concat(dfList).groupby(['product_id']).sum().reset_index().sort_values(by=['count'], ascending=False)
df

,product_id,count
5,1053,1736
2,1050,1480
0,1048,1023
1,1049,661
7,1055,625
4,1052,591
6,1054,457
8,1056,446
3,1051,186
11,1272,155
